# The 32-dim *Banana* example in the [GBS paper](http://proceedings.mlr.press/v118/jia20a.html)

In [1]:
import bayesfast as bf
import numpy as np
from threadpoolctl import threadpool_limits
threadpool_limits(1) # TODO: implement a bayesfast global thread controller

In [2]:
from scipy.stats import special_ortho_group

D = 32 # number of dims
Q = 0.01
lower = np.full(D, -15.) # lower bound of the prior
upper = np.full(D, 15.) # upper bound of the prior
bound = np.array((lower, upper)).T
diff = bound[:, 1] - bound[:, 0]
const = np.sum(np.log(diff)) # normalization of the flat prior

np.random.seed(0)
A = special_ortho_group.rvs(D) # random rotation of the bananas

def logp(x):
    x = x @ A.T
    return -np.sum((x[..., ::2]**2 - x[..., 1::2])**2 / Q + 
                   (x[..., ::2] - 1)**2, axis=-1) - const

def grad(x):
    x = x @ A.T
    _pfpx2i1 = 2 * (x[..., 1::2] - x[..., ::2]**2) / Q
    _pfpx2i = 2 * (x[..., ::2] - 1) - 2 * x[..., ::2] * _pfpx2i1
    res = np.empty_like(x)
    res[..., ::2] = _pfpx2i
    res[..., 1::2] = _pfpx2i1
    return -res @ A

In [3]:
bf.utils.random.set_generator(32) # set up the global random number generator
bf.utils.parallel.set_backend(8) # set up the global parallel backend
den = bf.DensityLite(logp=logp, grad=grad, input_size=D, input_scales=bound,
                     hard_bounds=True)
sample_trace = {'n_chain': 8, 'n_iter': 2500, 'n_warmup': 1000}
rec = bf.Recipe(density=den, sample={'sample_trace': sample_trace},
                post={'evidence_method': 'GBS'})

In [4]:
rec.run()

/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:183: RuntimeWarning:  CHAIN #1 : divide by zero encountered in log
  return np.sum(np.log(np.abs(self.to_original_grad(x_trans))),
/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:936: RuntimeWarning:  CHAIN #1 : invalid value encountered in true_divide
  _grad += self.to_original_grad2(x) / _tog
/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:183: RuntimeWarning:  CHAIN #2 : divide by zero encountered in log
  return np.sum(np.log(np.abs(self.to_original_grad(x_trans))),
/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:936: RuntimeWarning:  CHAIN #1 : divide by zero encountered in true_divide
  _grad += self.to_original_grad2(x) / _tog
/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:936: RuntimeWarning:  CHAIN #2 : invalid value encountered in true_divide
  _grad += self.to_original_grad2(x) / _tog
/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:183: RuntimeWarning:  CHAIN #3 : divide by zero encou

 CHAIN #1 : sampling proceeding [ 500 / 2500 ], last 500 samples used 41.88 seconds, while divergence encountered in 36 sample(s). (warmup)
 CHAIN #2 : sampling proceeding [ 500 / 2500 ], last 500 samples used 46.56 seconds. (warmup)
 CHAIN #3 : sampling proceeding [ 500 / 2500 ], last 500 samples used 53.21 seconds, while divergence encountered in 42 sample(s). (warmup)
 CHAIN #0 : sampling proceeding [ 500 / 2500 ], last 500 samples used 53.52 seconds. (warmup)
 CHAIN #6 : sampling proceeding [ 500 / 2500 ], last 500 samples used 53.86 seconds, while divergence encountered in 56 sample(s). (warmup)
 CHAIN #5 : sampling proceeding [ 500 / 2500 ], last 500 samples used 54.81 seconds. (warmup)
 CHAIN #4 : sampling proceeding [ 500 / 2500 ], last 500 samples used 55.66 seconds, while divergence encountered in 35 sample(s). (warmup)
 CHAIN #7 : sampling proceeding [ 500 / 2500 ], last 500 samples used 64.99 seconds, while divergence encountered in 34 sample(s). (warmup)
 CHAIN #1 : sampli

/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/glo

 ***** PostStep finished. ***** 



In [5]:
rec.get()._fields

('samples',
 'weights',
 'weights_trunc',
 'logp',
 'logq',
 'logz',
 'logz_err',
 'x_p',
 'x_q',
 'logp_p',
 'logq_q',
 'trace_p',
 'trace_q',
 'n_call')

In [6]:
rec.get().logz, rec.get().logz_err # fiducial value: logz = -127.364

(-127.25294056535556, 0.0523543560539916)